In [ ]:
# default_exp type_emb
# all_data

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Type embeddings

## Imports

In [ ]:
from socemb.read_data import *

In [ ]:
# export
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine

## Variables

In [ ]:
VECS_DIR = 'data/vecs/'

In [ ]:
LIMIT = 100_000

In [ ]:
VAR_TYPE = 'social'

In [ ]:
SUBREDDIT = 'askreddit'
T_1 = 2010
T_2 = 2020

In [ ]:
COMM_1 = 'askaconservative'
COMM_2 = 'asklibertarians'
YEAR = 2020

## Read comments

In [ ]:
fpath = get_fpath_subr_yr(SUBREDDIT, LIMIT, YEAR)

In [ ]:
comments = read_comm_csv(fpath)

In [ ]:
comments.info()

## Split data (inactive)

### Split in temporal bins

In [ ]:
comments = comments.assign(date = pd.to_datetime(
    comments['created_utc'],
    errors='coerce'
))

In [ ]:
comments['date'].dt.year.value_counts()

In [ ]:
comments = comments[comments.date.dt.year == TIME]

### Split by communities

In [ ]:
comments\
    .value_counts('subreddit')\
    .head(20)

In [ ]:
comments = comments.query('subreddit == "politics"')

In [ ]:
comments = comments.query('subreddit == "AskHistorians"')

## Pre-processing

In [ ]:
# export
def conv_to_lowerc(doc):
    return doc.lower()

In [ ]:
assert conv_to_lowerc('Test') == 'test'

In [ ]:
# export
def rm_punct(doc):
    return re.sub(r'[^\w\s]+', ' ', doc)

In [ ]:
assert rm_punct('No-punctuation!') == 'No punctuation '

In [ ]:
# export
def tokenize(doc):
    return doc.split()

In [ ]:
assert len(tokenize('These are three-tokens')) == 3

In [ ]:
# export
def detect_short_doc(doc, limit=10):
    if len(doc) >= limit:
        return False
    else:
        return True

In [ ]:
assert detect_short_doc(['oans', 'zwoa', 'drei'], 10) == True

In [ ]:
assert detect_short_doc(['oans', 'zwoa', 'drei', 'viere', 'fuenfe', 'sechse', 'simme', 'achte', 'neine', 'zehne'], 10) == False

In [ ]:
# export
def clean_docs(docs):
    docs_clean = docs\
        .apply(conv_to_lowerc)\
        .apply(rm_punct)\
        .apply(tokenize)\
        .where(lambda x : x.apply(detect_short_doc) == False)\
        .dropna()    
    return docs_clean

In [ ]:
docs_clean = clean_docs(comments['body'])

In [ ]:
docs_clean

### Lexeme-based

In [ ]:
docs_clean = docs_clean\
    .str.replace('anglo-saxon', 'anglosaxon')\
    .str.replace('anglo saxon', 'anglosaxon')

In [ ]:
docs_clean = docs_clean.str.replace('anglosaxon', 'Anglo-Saxon')

## Create corpus

In [ ]:
docs_clean

In [ ]:
# export
class Corpus:
    """An iterator that yields sentences (lists of str)."""
    def __init__(self, docs_clean):
        self.docs_clean = docs_clean

    def __iter__(self):
        for doc in self.docs_clean:
            yield doc

In [ ]:
corpus = Corpus(docs_clean)

## Train embeddings

In [ ]:
# export
def train_emb(corpus, MIN_COUNT=5, SIZE=300, WORKERS=8, WINDOW=3):
    model = Word2Vec(
        corpus, 
        min_count=MIN_COUNT,
        size=SIZE,
        workers=WORKERS, 
        window=WINDOW
    )
    return model

In [ ]:
%%time
model = train_emb(corpus)

In [ ]:
wv = model.wv

In [ ]:
wv['you'].shape

## Evaluate embeddings

In [ ]:
for index, word in enumerate(wv.index2word):
    if index == 10:
        break
    print(f"{index}/{len(wv.index2word)}: {word}")

In [ ]:
wv.most_similar('person', topn=20)

In [ ]:
lex_rel_1 = 'person'
lex_rel_2 = 'man'
lex_unrel = 'time'

In [ ]:
sim_rel = wv.similarity(lex_rel_1, lex_rel_2)
sim_rel

In [ ]:
sim_unrel_1 = wv.similarity(lex_rel_1, lex_unrel)
sim_unrel_1

In [ ]:
sim_unrel_2 = wv.similarity(lex_rel_2, lex_unrel)
sim_unrel_2

In [ ]:
assert sim_rel > sim_unrel_1

In [ ]:
assert sim_rel > sim_unrel_2

## Save vectors

In [ ]:
wv.save(f'{VECS_DIR}{SUBREDDIT}_{YEAR}_{LIMIT}.wv')

## Load vectors

In [ ]:
wv = KeyedVectors.load(f'{VECS_DIR}{SUBREDDIT}_{YEAR}_{LIMIT}.wv', mmap='r')

In the next cell, I need to to use `init_sims` to normalize the vectors. To do this, I need to save and load full models, not just the vector tables. Therefore, I need to train all models again.

In [ ]:
def load_model(SUBREDDIT, YEAR, LIMIT):
    model = KeyedVectors.load(f'data/vecs/{SUBREDDIT}_{YEAR}_{LIMIT}.wv', mmap='r')
    return model

In [ ]:
model_1 = KeyedVectors.load(f'data/vecs/year/2010.wv', mmap='r')

In [ ]:
model_1 = KeyedVectors.load(f'data/vecs/year/2020.wv', mmap='r')

## Measure distances

### For `1` pair of words

In [ ]:
vec_t1 = vecs_t1['person']

In [ ]:
vec_t2 = vecs_t2['person']

In [ ]:
assert vec_t1.shape[0] == 300

In [ ]:
assert vec_t2.shape[0] == 300

In [ ]:
cosine(vec_t1, vec_t2)

0.6213354766368866

### For full vocabulary

In [ ]:
def get_distances(model_1, model_2):
    dist_dict = {}
    # will need to replace `vecs_t1.vocab` with a general vocab
    for word in model_1.vocab:
        if word in model_2.vocab:
            dist_dict[word] = cosine(model_1[word], model_2[word])
        else:
            pass
    dist_dict_df = pd.DataFrame(
        data=dist_dict.items(), 
        columns=['word', 'dist']
    )\
        .sort_values('dist', ascending=False)
    return dist_dict_df

In [ ]:
dists = get_distances(model_1, model_2)

In [ ]:
dists.nlargest(20, 'dist')

,word,dist
87,that,1.053959
12078,mitch,0.956240
13711,stressed,0.938686
6964,pardons,0.927164
6551,adhd,0.923108
15113,2k,0.918534
6085,pardon,0.914406
5233,filibuster,0.913616
53,actually,0.912229
16466,kentucky,0.904231


### Inspect nearest neighbours

In [ ]:
lex = 'pardons'

In [ ]:
for model in [model_1, model_2]:
    for nb, dist in model.most_similar(lex, topn=10):
        print(f'{nb:<15}{model.vocab[nb].count:>6}')
    print('\n---\n')

statements        604
crimes           1746
allegations       224
arrests           134
convictions       114
appointees         66
witnesses         211
acts              634
charges           667
pardon           1009

---

dares               6
barrett            10
nominations         7
th                  5
maliciously         8
profiles            6
resisting          28
facist              6
delegates          20
byrd                5

---



### Temporal variation

In [ ]:
lex = 'trump'

In [ ]:
for model in [vecs_t1, vecs_t2]:
    for nb, dist in model.most_similar(lex, topn=10):
        print(nb)
    print('\n---\n')

lift
spark
tighten
stifle
weaken
override
alleviate
ramp
wash
enhance

---

biden
bernie
obama
himself
he
sanders
hillary
him
nixon
putin

---



### Social variation

In [ ]:
lex = 'quarantine'

In [ ]:
print(f'{COMM_1}\n---')
for nb, dist in model_1.most_similar(lex, topn=10):
    print(nb)

askaconservative
---
ppe
rain
bombings
port
aerosols
bullets
communications
missiles
weekly
2006


In [ ]:
print(f'{COMM_2}\n---')
for nb, dist in model_2.most_similar(lex, topn=10):
    print(nb)

asklibertarians
---
defense
defence
righteous
sufficiency
determination
isolate
legally
agency
permitted
explanatory


## Align models

In [ ]:
import numpy as np
import gensim

In [ ]:
smart_procrustes_align_gensim(model_1, model_2)

<ipython-input-226-3929bd79b83f>:33: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  old_arr = m.syn0norm


IndexError: index 18164 is out of bounds for axis 0 with size 17915

In [ ]:
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
	"""Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
	Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
		(With help from William. Thank you!)
	First, intersect the vocabularies (see `intersection_align_gensim` documentation).
	Then do the alignment on the other_embed model.
	Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
	Return other_embed.
	If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
	"""
	
	# patch by Richard So [https://twitter.com/richardjeanso) (thanks!) to update this code for new version of gensim
	base_embed.init_sims()
	other_embed.init_sims()

	# make sure vocabulary and indices are aligned
	in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

	# get the embedding matrices
	base_vecs = in_base_embed.syn0norm
	other_vecs = in_other_embed.syn0norm

	# just a matrix dot product with numpy
	m = other_vecs.T.dot(base_vecs) 
	# SVD method from numpy
	u, _, v = np.linalg.svd(m)
	# another matrix operation
	ortho = u.dot(v) 
	# Replace original array with modified one
	# i.e. multiplying the embedding matrix (syn0norm)by "ortho"
	other_embed.syn0norm = other_embed.syn0 = (other_embed.syn0norm).dot(ortho)
	return other_embed
	

In [ ]:
def intersection_align_gensim(m1,m2, words=None):
	"""
	Intersect two gensim word2vec models, m1 and m2.
	Only the shared vocabulary between them is kept.
	If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
	Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
	These indices correspond to the new syn0 and syn0norm objects in both gensim models:
		-- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
		-- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
	The .vocab dictionary is also updated for each model, preserving the count but updating the index.
	"""

	# Get the vocab for each model
	vocab_m1 = set(m1.vocab.keys())
	vocab_m2 = set(m2.vocab.keys())

	# Find the common vocabulary
	common_vocab = vocab_m1&vocab_m2
	if words: common_vocab&=set(words)

	# If no alignment necessary because vocab is identical...
	if not vocab_m1-common_vocab and not vocab_m2-common_vocab:
		return (m1,m2)

	# Otherwise sort by frequency (summed for both)
	common_vocab = list(common_vocab)
	common_vocab.sort(key=lambda w: m1.vocab[w].count + m2.vocab[w].count,reverse=True)

	# Then for each model...
	for m in [m1,m2]:
		# Replace old syn0norm array with new one (with common vocab)
		indices = [m.vocab[w].index for w in common_vocab]
		old_arr = m.syn0norm
		new_arr = np.array([old_arr[index] for index in indices])
		m.syn0norm = m.syn0 = new_arr

		# Replace old vocab dictionary with new one (with common vocab)
		# and old index2word with new one
		m.index2word = common_vocab
		old_vocab = m.vocab
		new_vocab = {}
		for new_index,word in enumerate(common_vocab):
			old_vocab_obj=old_vocab[word]
			new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
		m.vocab = new_vocab

	return (m1,m2)

In [ ]:
m1 = model_1
m2 = model_2

In [ ]:
vocab_m1 = set(m1.vocab.keys())
vocab_m2 = set(m2.vocab.keys())

In [ ]:
common_vocab = vocab_m1 & vocab_m2

In [ ]:
common_vocab = list(common_vocab)
common_vocab.sort(key=lambda w: m1.vocab[w].count + m2.vocab[w].count,reverse=True)

Note that for the new Gensim versions, calls for .index2word, .vocab, .syn0 and .syn0norm should be replaced with .wv.index2word, .wv.vocab, .wv.syn0 and .wv.syn0norm respectively.

In [ ]:
for m in [m1,m2]:
    # Replace old syn0norm array with new one (with common vocab)
    indices = [m.vocab[w].index for w in common_vocab]
    old_arr = m.syn0norm
    new_arr = np.array([old_arr[index] for index in indices])
    m.syn0norm = m.syn0 = new_arr

<ipython-input-255-02a3945973d7>:4: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  old_arr = m.syn0norm


IndexError: index 18164 is out of bounds for axis 0 with size 17915

<ipython-input-254-e3f9c0e261ee>:2: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  m.syn0norm = m.syn0 = new_arr
<ipython-input-254-e3f9c0e261ee>:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  m.syn0norm = m.syn0 = new_arr


In [ ]:
    # Replace old vocab dictionary with new one (with common vocab)
    # and old index2word with new one
    m.index2word = common_vocab
    old_vocab = m.vocab
    new_vocab = {}
    for new_index,word in enumerate(common_vocab):
        old_vocab_obj=old_vocab[word]
        new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
    m.vocab = new_vocab

In [ ]:
def align_gensim_models(models, words=None):
    """
    Returns the aligned/intersected models from a list of gensim word2vec models.
    Generalized from original two-way intersection as seen above.
    
    Also updated to work with the most recent version of gensim
    Requires reduce from functools
    
    In order to run this, make sure you run 'model.init_sims()' for each model before you input them for alignment.
    
    ##############################################
    ORIGINAL DESCRIPTION
    ##############################################
    
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocabs = [set(m.vocab.keys()) for m in models]

    # Find the common vocabulary
    common_vocab = reduce((lambda vocab1,vocab2: vocab1&vocab2), vocabs)
    if words: common_vocab&=set(words)

    # If no alignment necessary because vocab is identical...
    
    # This was generalized from:
    # if not vocab_m1-common_vocab and not vocab_m2-common_vocab and not vocab_m3-common_vocab:
    #   return (m1,m2,m3)
    if all(not vocab-common_vocab for vocab in vocabs):
        print("All identical!")
        return models
        
    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: sum([m.vocab[w].count for m in models]),reverse=True)
    
    # Then for each model...
    for m in models:
        
        # Replace old vectors_norm array with new one (with common vocab)
        indices = [m.vocab[w].index for w in common_vocab]
                
        old_arr = m.vectors_norm
                
        new_arr = np.array([old_arr[index] for index in indices])
        m.vectors_norm = m.syn0 = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        m.index2word = common_vocab
        old_vocab = m.vocab
        new_vocab = {}
        for new_index,word in enumerate(common_vocab):
            old_vocab_obj=old_vocab[word]
            new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
        m.vocab = new_vocab

    return models

In [ ]:
from functools import reduce

In [ ]:
model_1.init_sims()
model_2.init_sims()

In [ ]:
align_gensim_models([model_1, model_2])

IndexError: index 18164 is out of bounds for axis 0 with size 17915

## Analysis

### 'woke'

#### Nearest semantic neighbours

In [ ]:
model_2012.wv.most_similar(LEX)[:10]

In [ ]:
model_2013.wv.most_similar(LEX)[:10]

In [ ]:
model_2020.wv.most_similar(LEX)[:10]

#### Semantic distances

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import numpy as np

In [ ]:
woke_2012 = model_2012.wv.get_vector('woke')

In [ ]:
woke_2013 = model_2013.wv.get_vector('woke')

In [ ]:
woke_2020 = model_2020.wv.get_vector('woke')

In [ ]:
cosine_similarity(woke_2012.reshape(1, -1), woke_2020.reshape(1, -1))

In [ ]:
cosine_similarity(woke_2012.reshape(1, -1), woke_2013.reshape(1, -1))

### 'Anglo-Saxon'

#### diachronic

In [ ]:
model_anglo_saxon_2015 = model

In [ ]:
model_anglo_saxon_2015.wv.most_similar(LEX, topn=20)

In [ ]:
model_anglo_saxon_2020 = model

In [ ]:
model_anglo_saxon_2020.wv.most_similar(LEX, topn=20)

#### social

In [ ]:
model_anglo_saxon_politics = model

In [ ]:
model_anglo_saxon_politics.wv.most_similar(LEX, topn=20)

In [ ]:
model_anglo_saxon_historians = model

In [ ]:
model_anglo_saxon_historians.wv.most_similar(LEX, topn=20)

## Obsolete stuff

### Gensim preprocessing